In [ ]:
import holoviews as hv
hv.extension('bokeh')
hv.opts.defaults(hv.opts.Curve(width=500), 
                 hv.opts.Scatter(width=500, size=10), 
                 hv.opts.HLine(alpha=0.5, color='k', line_dash='dashed'))

In [ ]:
import numpy as np
import scipy.stats
# import corner

"""
Para instalar pyro sugiero usar conda y pip

    conda install -c pytorch pytorch pip
    pip install pyro-ppl

"""
import torch
import pyro

# Programación probabilística 

## Introducción

La programación probabilística (PP) es un nuevo paradigma que busca combinar los lenguajes de programación de propósito general con el modelamiento probabilístico.

El objetivo es hacer estadística y en particular inferencia Bayesiana usando las herramientas de ciencias de la computación. Como muestra el siguiente diagrama la PP corre en dos direcciones:

<a href="https://arxiv.org/abs/1809.10756"><img src="images/PP.png" width="500"></a>

El lenguaje Python tiene un ecosistema rico en frameworks y librerías de PP:

- [PyMC3](https://docs.pymc.io/notebooks/getting_started.html)
- [PyStan](https://pystan.readthedocs.io/en/latest/)
- [Edward](http://edwardlib.org/)
- [Pyro](http://pyro.ai)
- [emcee](http://dfm.io/emcee/current/)



En este tutorial aprenderemos a usar  la librería `pyro` 

1. Definir un modelo bayesiano en base a una verosimilitud y a un prior
1. Aplicar distintos algoritmos de MCMC sobre el modelo
1. Verificar la convergencia y analizar los posteriors

Usaremos como ejemplo una **regresión lineal Bayesiana**

## Regresión lineal bayesiana

Consideramos que tenemos $N$ tuplas $(x_i, y_i)$ donde $X$ es la variable independiente e $Y$ la dependiente

En una regresión queremos estimar $\mathbb{E}[Y|X]$ en base a un modelo paramétrico $Y = f_\theta(X)$. En este caso asumiremos un modelo lineal

$$
y_i = w x_i + b + \epsilon \quad i=1,2,\ldots,N
$$

donde queremos aprender los parámetros $\theta=(w, b)$ bajo el supuesto de que $p(\epsilon) = \mathcal{N}(0, \sigma^2)$

Luego 

$$
y \sim  \mathcal{N}(b + w x, \sigma^2)
$$

y por lo tanto la verosimilitud sería

$$
p(y|x,w,b,\sigma)  = \prod_{i=1}^N \mathcal{N}(b + w x_i, \sigma^2)
$$

A diferencia de una regresión "convencional" asumiremos que $w$, $b$ y $\sigma$ no son variables determinísticas sino **aleatorias** y por ende **tienen distribuciones**. Llamamos *prior* a la distribución de los parámetros previo a observar nuestros datos. 

En este caso particular asumiremos una distribución normal para los priors de $w$ y $b$

$$
p(b) = \mathcal{N}(\mu_b, \sigma_b^2)
$$

$$
p(w) = \mathcal{N}(\mu_w, \sigma_w^2)
$$

Lo que buscamos es el posterior de los parámetros $\theta$, es decir su distribución condicionado a nuestras observaciones $D$

$$
p(\theta|D) = \frac{p(D|\theta)p(\theta)}{p(D)}
$$

En este caso particular el posterior es

$$
p(w, b, \sigma|D) = \frac{p(D|w, b, \sigma) p(w) p(b) p(\sigma)}{\int p(D|w, b, \sigma) p(w) p(b) p(\sigma) dw db d\sigma}
$$

donde por simplicidad se asume que $p(w,b,\sigma) = p(w)p(b)p(\sigma)$, es decir que el prior no tiene correlaciones entre los parámetros.

:::{important}

Estimaremos este posterior en base a muestras utilizando MCMC

:::


La librería `pyro` utiliza `pytorch` como backend.

Esto significa que debemos definir o transformar nuestros datos a formato `torch.Tensor` para poder entregarlos a nuestros modelos. 

A continuación se generan los datos que utilizaremos a lo largo de este tutorial. 

In [ ]:
pyro.set_rng_seed(1234)

b_star, w_star, sigma_star, N = 1, 2.5, 1., 10
x = torch.randn(N)
y = b_star + w_star*x +  torch.randn(N)*sigma_star

x_test = torch.linspace(-3, 3, 100)

In [ ]:
hv.Scatter((x, y)).opts(color='k')

## Especificación del modelo generativo 

El modelo generativo es aquel que "produjo" los datos. Usualmente comienza con los hiperparámetros, continua en las variables latentes (priors) y termina en las variables observadas (verosimilitud). A continuación se muestra el diagrama de placas de la regresión lineal bayesiana con todas sus variables, parámetros e hiperparámetros

<img src="images/lin_reg_plate.png" width="600">


En `pyro` un modelo generativo es simplemente una función que define y relaciona las variables determinísticas y aleatorias que utilizaremos. Para crear una variable aleatoria se utiliza la primitiva

```python
pyro.sample(name, # El nombre de la variable (string)
            fn, # La distribución de la variable
            obs, # (Opcional) Los datos observados asociados a esta variable
            ...)
```

Los priors son variables aleatorias que no usan el argumento `obs`. En cambio, para escribir la verosimilitud, utilizamos el argumento `obs` para proporcionar los datos. Las distribuciones conocidas se pueden importar desde `pyro.distributions`. 

Para definir una variable determínistica se utiliza la primitiva

```python
pyro.deterministic(name, # El nombre de la variable (string)
                   value, # Transformación sobre otras variables del modelo
                   ...
                   )
```

A continuación se muestra la implementación del modelo de regresión lineal en `pyro`. Utilizaremos $\mu_w = \mu_b = 0$ y $\sigma_w = \sigma_b = 10.$

In [ ]:
HalfNormal?

In [ ]:
from pyro.distributions import Normal, HalfNormal

def model(x, y=None):
    w = pyro.sample("w", Normal(0.0, 10.0))
    b = pyro.sample("b", Normal(0.0, 10.0))
    s = pyro.sample("s", HalfNormal(1.0))
    with pyro.plate('datos', size=len(x)):
        f = pyro.deterministic('f', x*w + b)
        pyro.sample("y", Normal(f, s), obs=y)    
        

Durante la definición del modelo también utilizamos la primitiva

```python
pyro.plate(name, # El nombre del contexto (string)
           size=None, # El tamaño del dataset (int)
           device=None, # Se puede escoger entre cpu o gpu
           ...
           )
```
para crear una variable `y` que está condicionada al conjunto de variables `x`. Internamente `pyro.plate` también se hace cargo de paralelizar operaciones.

Un primer diagnóstico para asegurarnos de que nuestro modelo está definido correctamente es observar las muestras que están generando.

Podemos muestrear desde nuestro modelo utilizando

```python
pyro.infer.Predictive(model, # El modelo que definimos anteriormente
                      num_samples=None, # El número de muestras que deseamos generar
                      return_sites=(), # Las variables de las cuales deseamos muestrear
                      posterior_samples=None, # Opcional: Lo veremos más adelante
                      ...
                     )
```


In [ ]:
predictive = pyro.infer.Predictive(model, return_sites=("w", "b", "s", "f"), num_samples=2000)
prior_samples = predictive(x_test)

La variable `samples` contiene un diccionario cuyas llaves son las variables seleccionadas en `return_sites` y valores son tensores de 1000 elementos. 

En base a las muestras podemos construir histogramas o gráficos de densidad como se muestra a continuación. 

In [ ]:
joint = hv.Bivariate((prior_samples['w'], prior_samples['b']), 
                     kdims=['w', 'b']).opts(cmap='Blues', line_width=0, filled=True)

wmarginal, bmarginal = ((hv.Distribution(joint, kdims=[dim])) for dim in 'wb')
(joint) << bmarginal.opts(width=125) << wmarginal.opts(height=125)

La distribuciones estimadas a partir de las muestras son consistentes con los priors que escogimos.

Adicionalmente podemos estudiar el espacio de posibles modelos 

In [ ]:
p = []
for curve in prior_samples['f'][:100, :]:
    p.append(hv.Curve((x_test, curve)).opts(color='#30a2da', alpha=0.25))
hv.Overlay(p)

Esto se conoce como **distribución prior predictiva**

## MCMC con pyro

La maquinaria de MCMC en `pyro` se accede usando la función 

```python
pyro.infer.MCMC(kernel, # Un algoritmo muestreador, por ejemplo Metropolis
                num_samples, # Largo de la traza (sin contar burn-in)
                warmup_steps=None, # Cantidad de muestras iniciales a descartar
                initial_params=None, # (Opcional) Valores iniciales para la cadena
                num_chains=1, # Número de cadena (se pueden correr en paralelo)
                ... 
               )
``` 

Los principales métodos de `infer.MCMC` son

- `run()`: Realiza el muestreo y pobla las cadenas, espera los mismos argumentos que la función `model`
- `diagnostics()`: Retorna diagnósticos de la convergencia de la cadena (como Gelman-Rubin)
- `summary()`: Retorna una tabla con los momentos estadísticos de los parámetros
- `get_sample()`: Retorna la traza, es decir las muestras del posterior

El argumento más importante de `infer.MCMC` es el `kernel`. Actualmente hay dos disponibles: `HMC` ([Hamiltonian Monte Carlo](https://arxiv.org/abs/1312.0906)) y `NUTS` ([No-U turn sampler](https://arxiv.org/abs/1111.4246)). Ambos son muestreadores para parámetros continuos que utilizan información del gradiente para proponer transiciones.

En general, cada iteración de HMC/NUTS es más costosa con respecto a Metropolis-Hastings, pero en general se requieren menos iteraciones ya que converge más rápido al estado estacionario. Recomiendo revisar los siguientes [ejemplos animados](http://arogozhnikov.github.io/2016/12/19/markov_chain_monte_carlo.html) para tener una idea conceptual de la diferencia entre Metropolis y HMC.

NUTS es ampliamente considerado como el estado del arte en algoritmos de propuestas para paramétros continuos. Veamos a continuación como se muestrea usando MCMC y NUTS con `pyro`

In [ ]:
from pyro.infer import MCMC, NUTS

sampler = MCMC(kernel=NUTS(model, adapt_step_size=True), 
               num_chains=2, num_samples=1000, warmup_steps=100)

sampler.run(x, y)

Antes de usar el posterior es muy recomendable diagnosticar la adecuada convergencia de las cadenas. En primer lugar  podemos utilizar

In [ ]:
sampler.diagnostics()

De donde podemos resaltar que

- El número de muestras efectivo es alto (del orden de 50%)
- El estadísitco de Gelman Rubin es cercano a 1 para ambos parámetros
- No hubieron divergencias durante el muestro

Todo indicativos de una buena convergencia. También podemos obtener las trazas utilizando

In [ ]:
posterior_samples = sampler.get_samples()

A continuación se visualizan las trazas de `w`, `b` y `s`

In [ ]:
p1 = hv.Curve((posterior_samples['w']), 'Iteraciones', 'Traza', label='w')
p2 = hv.Curve((posterior_samples['b']), 'Iteraciones', 'Traza', label='b')
p3 = hv.Curve((posterior_samples['s']), 'Iteraciones', 'Traza', label='s')

hv.Overlay([p1, p2, p3]).opts(legend_position='top')

Como vimos la lección anterior la autocorrelación de la traza es una excelente herramienta para diagnosticar la correcta operación del algoritmo. En este caso la autocorrelación es

In [ ]:
posterior_samples.keys()

In [ ]:
def autocorrelation(theta_trace):
    thetas_norm = (theta_trace-np.mean(theta_trace))/np.std(theta_trace)
    rho = np.correlate(thetas_norm, 
                       thetas_norm, mode='full')
    return rho[len(rho) // 2:]/len(theta_trace)

rho = {}
for param in ['w', 'b', 's']:
    rho[param] = autocorrelation(posterior_samples[param].numpy())

In [ ]:
p = []
for key, value in rho.items():
    p.append(hv.Curve((value), 'Retardo', 'Traza', label=key))

hv.Overlay(p)

:::{note}

Para ambos parametros la autocorrelación decrece rapidamente y se mantiene en torno a cero

:::

Las métricas y diagnósticos nos indican que el algoritmo MCMC convergió al estado estacionario. Por lo tanto podemos  inspeccionar y utilizar el posterior para nuestro modelo de regresión lineal sin preocupaciones.

En primer lugar podemos obtener algunos estadísticos de los posterior con

In [ ]:
sampler.summary(prob=0.9)

También podemos explorar los posterior de `w` y `b` con estimadores de densidad basados en las muestras de la traza como se muestra a continuación

In [ ]:
joint = hv.Bivariate((posterior_samples['w'], posterior_samples['b']), 
                     kdims=['w', 'b']).opts(cmap='Blues', line_width=0, filled=True)

wmarginal, bmarginal = ((hv.Distribution(joint, kdims=[dim])) for dim in 'wb')
(joint) << bmarginal.opts(width=125) << wmarginal.opts(height=125)

:::{note}

Claramente el posterior $p(\theta| \mathcal{D})$ se ha desplazado con respecto al prior $p(\theta)$ que vimos anteriormente

:::

Ahora que tenemos el posterior de los parámetros podemos usarlo para calcular la **distribución posterior predictiva** en función de nuevos datos $\textbf{x}$

$$
p(\textbf{y}|\textbf{x}, \mathcal{D}) = \int p(\textbf{y}|\textbf{x},\theta) p(\theta| \mathcal{D}) \,d\theta 
$$

donde en este caso $\theta = (w, b, \sigma)$ y se asume que $y$ es condicionalmente independiente de  $\mathcal{D}$ dado que conozco $\theta$.

La parte más difícil era estimar $p(\theta| \mathcal{D})$ el cual ya tenemos gracias a MCMC. Para obtener muestras del posterior predictivo podemos nuevamente usar la clase `predictive` pero ahora le entregramos las muestras del posterior como argumento

In [ ]:
predictive = pyro.infer.Predictive(model, return_sites=(["f"]), 
                                   posterior_samples=posterior_samples)
samples = predictive(x_test)

En la siguiente figura aparecen los datos como puntos negros y 100 muestras del posterior predictivo (lineas azules)

In [ ]:
data = hv.Scatter((x, y), label='datos').opts(color='k')
p = []
for curve in samples['f'][:100, :]:
    p.append(hv.Curve((x_test, curve)).opts(color='#30a2da', alpha=0.1))
hv.Overlay(p) * data

:::{important}

Nuestro modelo bayesiano nos retorna una distribución de soluciones

:::

También podemos presentar la distribución graficamente usando estadísticos como se muestra a continuación

In [ ]:
p5, p50, p95 = samples['f'].quantile(torch.tensor([0.05, 0.5, 0.95]), axis=0)
line = hv.Curve((x_test, p50), label='mediana')
shade = hv.Spread((x_test, p50, p95-p5), label='95% CI').opts(color='#30a2da', alpha=0.5)
hv.Overlay([line, shade, data]).opts(legend_position='bottom_right')

:::{important}

Con el posterior podemos estudiar no sólo la solución más probable sino también el rango de las soluciones. El rango o ancho de la distribución está relacionado a la incertidumbre de nuestro modelo y observaciones (ruido)

:::